# RAG graph

In [1]:
pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain transformers faiss-cpu networkx huggingface-hub

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline, GPT4All, HuggingFaceHub
from langchain.chains import LLMChain, RetrievalQA
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.docstore import InMemoryDocstore
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

import faiss
import numpy as np
import re
import networkx as nx
import torch
import logging
from datetime import datetime
import json

## Retrieval Part (to be changed with the new one)

### Knowledge Base (KB) Graph Structure

- **Nodes**:
  - **Machine Nodes**: Represent machines with attributes like `model` and `manufacturer`.
  - **Base KPI Nodes**: Directly measured and stored in the database, such as:
    - `WorkingTime`: Measures time actively working.
    - `IdleTime`: Measures time idle but available.
    - `OfflineTime`: Measures time offline and unavailable.
  - **Non-Base KPI Nodes**: KPIs that require calculation based on Base KPIs, including:
    - `TotalAvailableTime`: Sum of `WorkingTime` and `IdleTime`.
    - `UtilizationRate`: Percentage of active working time as a function of availability.

- **Edges**:
  - **Machine-KPI Relationships**:
    - Each machine node is linked to Base KPI nodes with a `"measures"` edge.
  - **KPI Dependencies**:
    - Non-Base KPIs are connected to their required Base KPIs with `"depends_on"` edges, allowing dynamic calculation.

This organization helps the RAG agent understand:
- **Direct measurements**: From machine to Base KPIs.
- **Dependency hierarchy**: For calculated KPIs using other KPIs as input.

In [3]:
# Initialize a directed graph
G = nx.DiGraph()

# Add Machine Nodes with Attributes
G.add_node("LaserCutter_1", node_type="Machine", model="LC-200", manufacturer="Brand A")
G.add_node("LaserCutter_2", node_type="Machine", model="LC-300", manufacturer="Brand B")

# Add KPI Nodes with Descriptions and Normal Ranges
# Base KPIs (no fromula, directly saved in the DB)
G.add_node(
    "WorkingTime",
    node_type="Base KPI",
    description="Time actively working",
    unit="seconds",
    normal_min=6,
    normal_max=10)

G.add_node("IdleTime",
           node_type="Base KPI",
           description="Time idle but available",
           unit="seconds",
           normal_min=1,
           normal_max=4)

G.add_node("OfflineTime",
           node_type="Base KPI",
           description="Time offline and not available",
           unit="seconds",
           normal_min=0,
           normal_max=2)


# New KPIs (calculation needed)
# Define new KPI nodes
G.add_node(
    "TotalAvailableTime",
    node_type="NB_KPI",
    description="Total time available for work (including working and idle time)",
    unit="seconds",
    normal_min=7,
    normal_max=14,
    formula="WorkingTime + IdleTime"
)

G.add_node(
    "UtilizationRate",
    node_type="NB_KPI",
    description="Percentage of time actively working while available",
    unit="percentage",
    normal_min=60,
    normal_max=90,
    formula="(WorkingTime / (WorkingTime + IdleTime)) * 100"
)

# Add Directed Relationships (Edges) Between Machines and KPIs
G.add_edge("LaserCutter_1", "WorkingTime", relationship="measures")
G.add_edge("LaserCutter_1", "IdleTime", relationship="measures")
G.add_edge("LaserCutter_1", "OfflineTime", relationship="measures")

G.add_edge("LaserCutter_2", "WorkingTime", relationship="measures")
G.add_edge("LaserCutter_2", "IdleTime", relationship="measures")
G.add_edge("LaserCutter_2", "OfflineTime", relationship="measures")

G.add_edge("LaserCutter_1", "TotalAvailableTime", relationship ="measures")
G.add_edge("LaserCutter_1", "UtilizationRate", relationship ="measures")
G.add_edge("LaserCutter_2", "TotalAvailableTime", relationship ="measures")
G.add_edge("LaserCutter_2", "UtilizationRate", relationship ="measures")


# Add Directed Relationships (Edges) Between KPIs (base and calculated ones)
G.add_edge("TotalAvailableTime", "WorkingTime", relationship="depends_on")
G.add_edge("TotalAvailableTime", "IdleTime", relationship="depends_on")

G.add_edge("UtilizationRate", "WorkingTime", relationship="depends_on")
G.add_edge("UtilizationRate", "IdleTime", relationship="depends_on")


### Create embeddings for each node

For each node, a textual description is created.

In [4]:
# Function to generate descriptions automatically
def generate_descriptions(graph):
    descriptions = {}

    for node in graph.nodes(data=True):
        node_name, attributes = node
        node_type = attributes.get("node_type")

        if node_type == "Machine":
            # Find KPIs measured by this machine
            kpis = [
                target for source, target, data in graph.edges(data=True)
                if source == node_name and data["relationship"] == "measures"
            ]
            kpi_list = ", ".join(kpis)

            # Generate description using template for machines
            descriptions[node_name] = (
                f"it is a Machine. Model {attributes.get('model')}, "
                f"manufactured by {attributes.get('manufacturer')}. It has the following KPIs: {kpi_list}."
            )

        elif node_type == "Base KPI":
            # Find machines that measure this Base KPI
            machines = [
                source for source, target, data in graph.edges(data=True)
                if target == node_name and data["relationship"] == "measures"
            ]
            machine_list = ", ".join(machines)

            # Generate description using template for Base KPIs
            descriptions[node_name] = (
                f"it is a base KPI. It measures {attributes.get('description')} in {attributes.get('unit')}. "
                f"Reference range is {attributes.get('normal_min')} to {attributes.get('normal_max')}. "
                f"Used by machines: {machine_list}."
            )

        elif node_type == "NB_KPI":
            # Find dependencies for this Non-Base KPI
            dependencies = [
                target for source, target, data in graph.edges(data=True)
                if source == node_name and data["relationship"] == "depends_on"
            ]
            dependency_list = ", ".join(dependencies)

            # Generate description using template for Non-Base KPIs
            descriptions[node_name] = (
                f"it is a non-base KPI. It measures {attributes.get('description')} in {attributes.get('unit')}. "
                f"Reference range is {attributes.get('normal_min')} to {attributes.get('normal_max')}. "
                f"Calculated from: {dependency_list}. Formula: {attributes.get('formula')}."
            )

    return descriptions

# Generate and print descriptions for all nodes
generated_descriptions = generate_descriptions(G)
descriptions = []


for node, desc in generated_descriptions.items():
    print(f"{node}: {desc}")
    descriptions.append({node: desc})

LaserCutter_1: it is a Machine. Model LC-200, manufactured by Brand A. It has the following KPIs: WorkingTime, IdleTime, OfflineTime, TotalAvailableTime, UtilizationRate.
LaserCutter_2: it is a Machine. Model LC-300, manufactured by Brand B. It has the following KPIs: WorkingTime, IdleTime, OfflineTime, TotalAvailableTime, UtilizationRate.
WorkingTime: it is a base KPI. It measures Time actively working in seconds. Reference range is 6 to 10. Used by machines: LaserCutter_1, LaserCutter_2.
IdleTime: it is a base KPI. It measures Time idle but available in seconds. Reference range is 1 to 4. Used by machines: LaserCutter_1, LaserCutter_2.
OfflineTime: it is a base KPI. It measures Time offline and not available in seconds. Reference range is 0 to 2. Used by machines: LaserCutter_1, LaserCutter_2.
TotalAvailableTime: it is a non-base KPI. It measures Total time available for work (including working and idle time) in seconds. Reference range is 7 to 14. Calculated from: WorkingTime, IdleT

In [5]:
# Step 1: Convert node descriptions into Langchain Document objects
documents = []

# Iterate over the list of dictionaries in `descriptions`
for item in descriptions:
    for node, desc in item.items():  # Extract the key-value pair from each dictionary
        # Include the node name in the page content to make it explicit
        document_text = f"{node}: {desc}"
        documents.append(Document(page_content=document_text, metadata={"node": node}))

# Step 2: Generate embeddings for each node description using HuggingFace
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = [embedding_model.embed_query(doc.page_content) for doc in documents]

# Step 3: Create FAISS index and add the embeddings
dimension = len(embeddings[0])
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(np.array(embeddings))

# Step 4: Create `index_to_docstore_id` and a `docstore`
index_to_docstore_id = {i: str(i) for i in range(len(documents))}
docstore = InMemoryDocstore(({str(i): doc for i, doc in enumerate(documents)}))

# Step 5: Create FAISS vector store with HuggingFace embeddings and the node documents
vector_store = FAISS(
    embedding_function=embedding_model,
    index=faiss_index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)


/var/folders/sp/6tltq_zj57n0ys2h5jz2ch3h0000gn/T/ipykernel_60396/2713839655.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [6]:
# Verification: Print documents to verify the KPI names are included in the content
for doc in documents:
    print(doc.page_content, doc.metadata)

# Verification: Print embedding lengths
for i, embedding in enumerate(embeddings):

  embedding = np.array(embedding)
  print(embedding.shape)
  print(f"Embedding {i}: Length = {len(embedding)}")

# Verification: Print FAISS index size
print(f"FAISS index size: {faiss_index.ntotal}")

LaserCutter_1: it is a Machine. Model LC-200, manufactured by Brand A. It has the following KPIs: WorkingTime, IdleTime, OfflineTime, TotalAvailableTime, UtilizationRate. {'node': 'LaserCutter_1'}
LaserCutter_2: it is a Machine. Model LC-300, manufactured by Brand B. It has the following KPIs: WorkingTime, IdleTime, OfflineTime, TotalAvailableTime, UtilizationRate. {'node': 'LaserCutter_2'}
WorkingTime: it is a base KPI. It measures Time actively working in seconds. Reference range is 6 to 10. Used by machines: LaserCutter_1, LaserCutter_2. {'node': 'WorkingTime'}
IdleTime: it is a base KPI. It measures Time idle but available in seconds. Reference range is 1 to 4. Used by machines: LaserCutter_1, LaserCutter_2. {'node': 'IdleTime'}
OfflineTime: it is a base KPI. It measures Time offline and not available in seconds. Reference range is 0 to 2. Used by machines: LaserCutter_1, LaserCutter_2. {'node': 'OfflineTime'}
TotalAvailableTime: it is a non-base KPI. It measures Total time availab

In [7]:
retriever = vector_store.as_retriever()
test_query = "Get info about Laser Cutter Machines"
results = retriever.invoke(test_query)
for result in results:
    print(f"Retrieved:\n {result.page_content}")

Retrieved:
 LaserCutter_2: it is a Machine. Model LC-300, manufactured by Brand B. It has the following KPIs: WorkingTime, IdleTime, OfflineTime, TotalAvailableTime, UtilizationRate.
Retrieved:
 LaserCutter_1: it is a Machine. Model LC-200, manufactured by Brand A. It has the following KPIs: WorkingTime, IdleTime, OfflineTime, TotalAvailableTime, UtilizationRate.
Retrieved:
 WorkingTime: it is a base KPI. It measures Time actively working in seconds. Reference range is 6 to 10. Used by machines: LaserCutter_1, LaserCutter_2.
Retrieved:
 OfflineTime: it is a base KPI. It measures Time offline and not available in seconds. Reference range is 0 to 2. Used by machines: LaserCutter_1, LaserCutter_2.


In [8]:
print(torch.cuda.is_available())  # Should return True if GPU is available


model = ChatOpenAI(
    base_url = 'http://localhost:11434/v1',
    temperature = 0, #quanto il modello si deve attenere al prompt
    api_key = 'not-needed',
    model_name = 'mistral'
)

prompt = ChatPromptTemplate.from_template("Tell me what a {argument} is.")
output_parser = StrOutputParser()
chain = prompt | model | output_parser

answer = chain.invoke({"argument" : "KPI"})

print(answer)

False
 A Key Performance Indicator (KPI) is a measurable value that demonstrates how effectively a company or department is achieving key business objectives. KPIs are used to evaluate success at both the macro and micro levels, providing organizations with a way to track progress towards strategic goals and identify areas for improvement. They can be quantitative or qualitative, and are often expressed as ratios, percentages, or raw numbers. Examples of KPIs include revenue growth rate, customer satisfaction score, employee turnover rate, and website traffic.


# Generative Part

### Steps of elaboration of the user query:

Check if the query can be answered (informational query) or if there is the need to retrieve historical data (action query):
- if the query is informational, the model just answers it based on the provided context
- if the query is action, the model has to extract specific elements to be able to contact the KPI calculation engine.

The elements needed are:
- kpi name
- machine name
- the query dateframe (single date, range of dates, aggregation period)
- operation (optional)


In [13]:
import json

# Define the different prompt temmplates
enough_info_prompt = ChatPromptTemplate.from_template(
    """
    You are an evaluator.  Your job is not to answer the query. Your job is to decide whether historical data is needed to answer the query. 
    Explain your reasoning and conclude with a final answer: respond 'yes' if historical data is needed, otherwise 'no'. 
    You must strictly adhere to this format. 
    Context: {context}. Question: {query}. 
    Answer:
    """
)

informational_query_prompt = ChatPromptTemplate.from_template(
    """
    Please answer the following question using the information in the provided context. 
    Your answer should be direct and concise, focusing specifically on addressing the question. 
    If the question asks for additional details, provide only the specific information requested. 
    Do not introduce information or explanations beyond what is directly asked for in the question. 
    Context: {context}. Question: {query}. 
    Answer:
    """
)

action_query_prompt = ChatPromptTemplate.from_template(
"""
    Analyze the provided query to extract specific details. Follow these steps precisely:

    Step 1: Determine the Timeframe 
    Identify the timeframe mentioned in the query. Classify it into one of the following categories:

    1. Specific Date:
    - A single, specific point in time. Examples include:
        - "on September 15th"
        - "on 2023-09-15"
        - "yesterday"
        - "two days ago"
    
    2. Range of Dates:
    - A continuous range that has a clear start and end date. Examples include:
        - "between September 1st and September 15th"
        - "over the past week"
        - "from July to August"
    - Note: A range always involves a start and an end date, covering all the days in between.

    3. Aggregation Period:
    - A period that indicates a repeating aggregation, usually associated with a specific frequency or regular interval, to group data. Examples include:
        - "monthly between July and September" (meaning data grouped month by month)
        - "weekly in the last month" (meaning data grouped week by week over the past month)
    - Note: An aggregation period focuses on dividing the timeframe into repeated intervals (e.g., weekly, monthly) for aggregated analysis. Only classify it as an aggregation period if **explicit** mention of terms like "monthly", "weekly", or similar are found in the query.

    Step 2: Identify the Operation
    Check if the query mentions any of the following operations:
    - sum, avg, max, min.
    If no operation is mentioned, leave this field as 'null'.

    Step 3: Extract the Following Details:
    Extract and organize the information as described below:
    - For a Specific Date:
        - Provide it in the `start_range` field in YYYY-MM-DD format.
    - For a Date Range:
        - Provide both `start_range` and `end_range` fields in YYYY-MM-DD format.
    - For an Aggregation Period:
        - Provide `start_range` and `end_range` fields in YYYY-MM-DD format.
        - Additionally, include the `aggregation` field (e.g., 'monthly', 'weekly', or 'daily').
    - 'operation': Specify the operation mentioned (e.g., 'sum,' 'avg,' 'max,' 'min'). If none, set this field to 'null'.
    - 'KPI_name': Identify the key performance indicator mentioned. 
    - 'machine_name': Determine the machine referred to in the query.

    Return the extracted information in a structured format (json).
    Today's date is {current_date} for reference.

    Query: {query}.
    Context: {context}.
    Answer
"""
)


# Define the different pipelines
chain1 = (
    {
        'context': RunnablePassthrough(), 
        'query': RunnablePassthrough()  
    }
    | enough_info_prompt  
    | model  
    | StrOutputParser()  
)

chain2 = (
    {
        'context': RunnablePassthrough(),  
        'query': RunnablePassthrough()  
    }
    | informational_query_prompt 
    | model   
    | StrOutputParser()  
)

chain3 = (
    {
        'context': RunnablePassthrough(), 
        'query': RunnablePassthrough(), 
        'current_date':  RunnablePassthrough()
    }
    | action_query_prompt 
    | model   
    | StrOutputParser()  
)

In [21]:
def extract_json_from_llm_response(response):
    # Trova tutte le parti tra parentesi graffe
    matches = re.findall(r'\{.*?\}', response, re.DOTALL)
    
    if not matches:
        return {}
    
    # Considera solo il primo match, assumendo che sia il contenuto da convertire in JSON
    content = matches[0]
    
    # Rimuove i commenti dopo le virgole
    lines = content.splitlines()
    cleaned_lines = []
    for line in lines:
        # Rimuove tutto dopo la virgola, se presente
        cleaned_line = re.sub(r',.*', '', line).strip()
        if cleaned_line:
            cleaned_lines.append(cleaned_line)
    
    # Crea un dizionario filtrando solo gli elementi desiderati
    desired_keys = ["KPI_name", "machine_name", "start_range", "end_range", "aggregation", "operation"]
    result = {key: None for key in desired_keys}
    
    for line in cleaned_lines:
        for key in desired_keys:
            if line.startswith(f'"{key}"'):
                # Estrae il valore dopo i due punti
                value = line.split(':', 1)[-1].strip().strip('"')
                result[key] = value
                break
    
    return result


In [22]:
# Function to call the RAG pipeline
def steps(query, context, date):
    # Step 1: initial prompt (chian1)
    input_data = {"context": context, "query": query}
    response_1 = chain1.invoke(input_data)
    #print("\nResponse: {}".format(response_1))

    # Step 2: Check the first two tokens 
    first_two_tokens = response_1.strip().split()[:1]

    # If the two tokens are "No,", the query is informational
    if " ".join(first_two_tokens) == "No,":
        response_2 = chain2.invoke(input_data)
        return response_2 
    
    # Otherwise, the query is action
    else:
        input_data = {"context": context, "query": query, "current_date": date}
        response_3 = chain3.invoke(input_data)

        response_3 = extract_json_from_llm_response(response_3)
        return response_3

# Get the current date
current_date = datetime.now().strftime("%Y-%m-%d")

In [73]:
# Examples of informative queries
queries_info = [
    "How many Laser Cutter machines are there?",
    "Show me the available KPIs for the Laser Cutter machines.",
    "Is there a KPI for material cost?",
    "Is there a KPI for that measures the percentage of time working?",
]

for query in queries_info:
    # Retrieve relevant context using the retriever
    context_docs = retriever.invoke(query)
    context = " ".join([doc.page_content for doc in context_docs])

    print(f"\nQuery: {query}")
    # print(f"\nContext: {context}")

    response = steps(query, context, current_date)
    
    print("\nResponse: {}".format(response))
    print("\n")
    print("-" * 12)
    print("\n")


Query: How many Laser Cutter machines are there?

Response: 2 Laser Cutter machines


------------



Query: Show me the available KPIs for the Laser Cutter machines.

Response:  The available KPIs for the Laser Cutter machines are WorkingTime, IdleTime, OfflineTime, TotalAvailableTime, and UtilizationRate.


------------



Query: Is there a KPI for material cost?

Response:  No, the provided context does not mention any Key Performance Indicator (KPI) specifically for material cost.


------------



Query: Is there a KPI for that measures the percentage of time working?

Response:  Yes, the KPI 'WorkingTime' measures the time actively working in seconds and it is a base KPI. However, if you are looking for a KPI that provides the percentage of time working, then 'UtilizationRate' would be the appropriate choice as it calculates the percentage of time actively working while available.


------------




In [23]:
# Examples of action queries
queries_action = [
    "What was the utilization rate for LaserCutter_1 the 2 of November?",
    "What is the utilization rate for LaserCutter_1 yesteday?",
    "What was the working time for LaserCutter_1 over the past week?",
    "What is the value of the total avaiable time for Laser cutter 1 in the month of agust?",
    "What is the average of utilization rate for laser cutter 2 between september and october?",
    "Provide the monthly sum of time idle but available of laser cutter 2 from July to September."
]

for query in queries_action:
    # Retrieve relevant context using the retriever
    context_docs = retriever.invoke(query)
    context = " ".join([doc.page_content for doc in context_docs])

    print(f"\nQuery: {query}")
    # print(f"\nContext: {context}")

    response = steps(query, context, current_date)
    
    print("\nResponse: {}".format(response))
    print("\n")
    print("-" * 12)
    print("\n")


Query: What was the utilization rate for LaserCutter_1 the 2 of November?

Response: {'KPI_name': 'UtilizationRate', 'machine_name': 'LaserCutter_1', 'start_range': '2024-11-02', 'end_range': 'null', 'aggregation': 'null', 'operation': 'null'}


------------



Query: What is the utilization rate for LaserCutter_1 yesteday?

Response: {'KPI_name': 'UtilizationRate', 'machine_name': 'LaserCutter_1', 'start_range': '2024-11-22', 'end_range': 'null', 'aggregation': 'null', 'operation': 'null'}


------------



Query: What was the working time for LaserCutter_1 over the past week?

Response: {'KPI_name': 'WorkingTime', 'machine_name': 'LaserCutter_1', 'start_range': '2024-11-16', 'end_range': '2024-11-22', 'aggregation': 'null', 'operation': 'null'}


------------



Query: What is the value of the total avaiable time for Laser cutter 1 in the month of agust?

Response: {'KPI_name': 'TotalAvailableTime', 'machine_name': 'LaserCutter_1', 'start_range': '2024-08-01', 'end_range': '2024-08-

# Elaboration steps of KPI engine output

For the different types of action queries we have identified, there are two possible types of output:
-	Single Value Response: If the query yields a single value, both the query and the calculated value are passed to the model and it generates the response directly.
-	List of Values Response: If the query yields a list of values, a table is used to present the data.

In [75]:
# Different prompts for different types of outputs
single_value_prompt = ChatPromptTemplate.from_template(
    """
    Please answer the following question using the provided value and the given unit of measurement. 
    Your answer should be direct and concise, focusing specifically on addressing the question. 
    If the question asks for additional details, provide only the specific information requested. 
    Do not introduce information or explanations beyond what is directly asked for in the question. 
    Value: {value}. Unit of measurement: {unit}. Query: {query}. 
    Answer:
    """
)

table_prompt = ChatPromptTemplate.from_template(
    """
    Create a brief and concise introductory statement that describes the table’s content based on the given query. Use just one statement. 
    
    Example:
    - Query: What was the working time for LaserCutter_1 over the past week?
    - Output: Below is the working time for LaserCutter_1 over the past week:
    Query: {query}.
    Answer:
    """
)

# Different chains 
chain4 = (
    {
        'query': RunnablePassthrough(), 
        'value': RunnablePassthrough(),
        'unit': RunnablePassthrough(),
    }
    | single_value_prompt 
    | model   
    | StrOutputParser()  
)

chain5 = (
    {
        'query': RunnablePassthrough()
    }
    | table_prompt  
    | model   
    | StrOutputParser()  
)

In [76]:


def response_creation(query, kpi_response):
    # Check the KPI engine output

    if "value" in kpi_response:
        # Single value
        kpi_value = kpi_response["value"]
        unit = kpi_response["unit"]

        input_data = {"value": kpi_value, "unit": unit, "query": query}
        response_4 = chain4.invoke(input_data)
        return response_4

    elif "values" in kpi_response:
        # Multiple values
        values = kpi_response["values"]
        unit = kpi_response["unit"]
        kpi_name = kpi_response["kpi_name"]

        operation = kpi_response["operation"]

        # Determine the column header based on the operation
        if operation:
            column_name = f"{operation.capitalize()} Value"
        else:
            column_name = "Value"
        
        aggregation = kpi_response.get("aggregation")

        # Determine the column headers based on the aggregation
        if aggregation:
            date_columns = "| From         | To           |"
        else:
            date_columns = "| Date         |"

        # Constructing the Markdown table
        table_header = f"{date_columns} {column_name} |\n"
        table_separator = "|--------------" * (2 if aggregation else 1) + "|-------------------|\n"

        table_rows = ""
        if aggregation:
            # Use start_date and end_date for each value entry
            for entry in values:
                table_rows += f"| {entry['start_date']} | {entry['end_date']} | {entry['value']}{unit} |\n"
        else:
            # Use date and value fields for each row
            for entry in values:
                table_rows += f"| {entry['date']} | {entry['value']} {unit} |\n"

        # Combine header, separator, and rows to create the full markdown table
        table_markdown = f"{table_header}{table_separator}{table_rows}"

        #print(table_markdown)

        # Let the model generate an introduction to the data
        input_data = {"query": query}
        response_5 = chain5.invoke(input_data)
        complete_response = f"{response_5}\n\n{table_markdown}"

        return complete_response

In [77]:
examples = [
    # Example 1: Single Value Response
    (
        "What is the average temperature recorded by Sensor_X during October?",
        {
            "machine_name": "Sensor_X",
            "kpi_name": "AverageTemperature",
            "value": 25.3,
            "unit": "°C"
        }
    ),
    # Example 2: Multiple Values Response (Aggregated Data)
    (
        "Provide the weekly production rates for Factory_1 over the past month.",
        {
            "machine_name": "Factory_1",
            "kpi_name": "WeeklyProductionRate",
            "operation": "sum",
            "aggregation": True,
            "values": [
                {"start_date": "2024-11-01", "end_date": "2024-11-07", "value": 1500},
                {"start_date": "2024-11-08", "end_date": "2024-11-14", "value": 1600},
                {"start_date": "2024-11-15", "end_date": "2024-11-21", "value": 1400}
            ],
            "unit": "units"
        }
    ),
    # Example 3: Multiple Values Response (Daily Data)
    (
        "Show the utilization rate for Machine_Z over the past week.",
        {
            "machine_name": "Machine_Z",
            "kpi_name": "UtilizationRate",
            "operation": None,
            "aggregation": None,
            "values": [
                {"date": "2024-11-02", "value": 72.4},
                {"date": "2024-11-03", "value": 75.8},
                {"date": "2024-11-04", "value": 78.1},
                {"date": "2024-11-05", "value": 80.3},
                {"date": "2024-11-06", "value": 74.5},
                {"date": "2024-11-07", "value": 82.1}
            ],
            "unit": "%"
        }
    ),
]


# Loop through each example case and generate the response
for index, (query, kpi_response) in enumerate(examples):
    print(f"\nExample {index + 1}:")
    try:
        print(f"Query: {query}")
        definitive_response = response_creation(query, kpi_response)
        print(definitive_response)
    except Exception as e:
        print(f"An error occurred: {e}")


Example 1:
Query: What is the average temperature recorded by Sensor_X during October?
 The average temperature recorded by Sensor_X during October was 25.3 degrees Celsius.

Example 2:
Query: Provide the weekly production rates for Factory_1 over the past month.
 Below is the weekly production rate for Factory_1 over the past month:

This statement provides a clear and concise summary of the table's content based on the given query. It lets the user know that they will be receiving the weekly production rates for Factory_1 over the past month from the table.

| From         | To           | Sum Value |
|--------------|--------------|-------------------|
| 2024-11-01 | 2024-11-07 | 1500units |
| 2024-11-08 | 2024-11-14 | 1600units |
| 2024-11-15 | 2024-11-21 | 1400units |


Example 3:
Query: Show the utilization rate for Machine_Z over the past week.
 Below is the utilization rate for Machine_Z over the past week:

This statement provides a clear and concise introduction to the table'